## ETL_user_reviews

In [1]:
#Importa las librerías pandas para manejo de datos, json para trabajar con archivos JSON 
# y ast para evaluar expresiones literales de Python.
import pandas as pd
from pandas import json_normalize # Importamos la función json_normalize de pandas para trabajar con datos JSON anidados.
import json as js 
import ast as ast

In [2]:
# Define la ruta del archivo JSON que se va a leer.
pathreviews = r'C:\Users\Sofi\Desktop\Henry\LABS\Proyecto_STEAM\Data\australian_user_reviews.json'

*Abrimos el archivo australian_user_reviews.json y lo guardamos en una variable de tipo lista*

In [3]:
# Creamos una lista vacia.
lista_reviews= [] 

# Abrimos el archivo JSON ubicado en 'pathreviews'. 
with open(pathreviews, encoding='utf-8') as file: 
    for line in file.readlines(): # Iteramos línea por línea en el archivo JSON.
        lista_reviews.append(ast.literal_eval(line)) # agregamos cada línea a la lista 'lista_reviews' después de evaluarla con ast.literal_eval() para convertir la cadena JSON en un objeto Python.

*Creamos un DataFrame 'df_reviews' a partir de la lista 'lista_reviews'.*\
*Realizamos una copia del DataFrame 'df_reviews' y la guardamos en 'df_reviews_clean' para realizar modificaciones sin alterar el original.*

In [4]:
# Creamos el Dataframe de la lista.
df_reviews  = pd.DataFrame(lista_reviews)
# Copia del df
df_reviews_clean = df_reviews.copy()

*Eliminamos la columna 'user_url' del DataFrame 'df_reviews_clean'.*

In [5]:
df_reviews_clean = df_reviews_clean.drop('user_url', axis=1)

In [6]:
df_reviews_clean

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...
25794,76561198306599751,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


*Desanidamos la columna 'reviews' del DataFrame 'df_reviews_clean', expandiendo los datos anidados en esa columna y reiniciando los índices del DataFrame resultante.*

In [7]:
#Desanidar la columna 'reviews'
df_expanded = df_reviews_clean.explode('reviews')

df_expanded.reset_index(drop=True, inplace=True)
#df_expanded.head(20)

*Normalizamos los datos de la columna 'reviews' del DataFrame 'df_expanded', convirtiendo los datos anidados en una estructura de DataFrame plana.*

In [8]:
df_reviews_normalized = json_normalize(df_expanded['reviews'])
#df_reviews_normalized.head(20)

*Combinamos los DataFrames 'df_expanded' (sin la columna 'reviews') y 'df_reviews_normalized' (con los datos normalizados de 'reviews') en un nuevo DataFrame llamado 'df_final'.*

In [9]:
#Combinar los DataFrames original y normalizado
df_final = pd.concat([df_expanded.drop(columns=['reviews']), df_reviews_normalized], axis=1)
#df_final.head(20)

*Creamos una lista de columnas a eliminar ('columns_to_drop').*\
*Eliminamos las columnas especificadas en 'columns_to_drop' del DataFrame 'df_final' y guardamos el resultado en 'df_reviews_clean'.*

In [10]:
#Lista de columnas a eliminar
columns_to_drop = ['funny', 'last_edited', 'helpful']

#Eliminar las columnas especificadas
df_reviews_clean = df_final.drop(columns=columns_to_drop, errors='ignore')

df_reviews_clean

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,True,:D


*Exportamos el DataFrame a un archivo CSV*

In [11]:
df_reviews_clean.to_parquet(r'C:\Users\Sofi\Desktop\Henry\LABS\Proyecto_STEAM\Data_Clean\user_reviews_clean.parquet', index=False)